### Do the imports
- If I am doing any processing of files in a folder I like to import os and glob
- import cellpose models for training of inferance
- import something to read your images 
- import libraries to save your results


In [1]:
import os
import glob
import sys

from cellpose import models
import napari
import torch
import nd2
import numpy as np

import skimage as ski

import sutils

Create the napari viewer to visualize images and mask

In [2]:
viewer = napari.Viewer()

### The segmentation results from the homework
<img src="files/cellpose/Data/easy_labels.png" width="750">

Find the image files to work with. Different path for Window, Mac, and Linux

In [3]:
filename = "files/Easy.tif"

In [4]:

easy = ski.io.imread(filename)
easy.shape

(512, 512)

Add the image to the napari viewer

In [5]:
viewer.layers.clear()
viewer.add_image(easy)

<Image layer 'easy' at 0x2e1c417c3a0>

### Setup the cellpose model

#### GPU setting
- on windows or linux with an NVIDIA GPU, set `gpu=True``
- on M1 or M2 mac, set `device=torch.device('mps')`
- on old Mac or Windows without GPU, set `gpu=False` -- this will be slower

#### Model
- cellpose has a number of pretrained models to use. This image looks like nuclei, so try the `nuclei` model. The `cyto` and `cyto2` models ofter work for nuclei.
- info about pretrained cellpose [models](https://cellpose.readthedocs.io/en/latest/models.html)


In [6]:
if sys.platform == 'darwin':
    d = torch.device('mps')
    model = models.Cellpose(gpu=False, device=d, model_type='cyto2')
else:
    # change gpu=True if on windows, and get rid of device
    model = models.Cellpose(gpu=True, model_type='cyto2')

### `model.eval`
Parameters
----------

The basic options:
- x : the image, can be a 2d numpy array, a list of numpy arrays,
or a 3d numpy array
- diameter : The approximate size of the object you are trying to segment
- channels :
    - [0, 0] for a grayscale image
    - [] 

Returns
-------
- masks : An array or list of arrays with segmenation labels/masks
- flows : A list of numpy arrays with fow

#### Try cellpose with no paramter changes

In [7]:

masks, flows, styles, diams = model.eval(easy, channels=[0, 0])

Show the results from cellpose a layers in napari

In [8]:
viewer.layers.clear()
viewer.add_image(easy)
viewer.add_labels(masks, name='cyto2, default settings')


<Labels layer 'cyto2, default settings' at 0x2e1d33dfe20>

The nuclei model is trained with a diameter of 17, but the default diameter is 30, so it will need to be adjusted.
See what happens when the diameter is changed. What diameter is the best?

In [9]:
masks2, flows2, styles2, diams2 = model.eval(easy, channels=[0, 0],
                                         diameter=12,
                                         #cellprob_threshold=0,
                                         #flow_threshold=.4
)

viewer.add_labels(masks2, name='cyto2, diameter=12')

<Labels layer 'cyto2, diameter=12' at 0x2e1c52a4040>

# Different model

In [10]:
if sys.platform == 'darwin':
    d = torch.device('mps')
    model = models.Cellpose(gpu=False, device=d, model_type='nuclei')
else:
    # change gpu=True if on windows, and get rid of device
    model = models.Cellpose(gpu=True, model_type='nuclei')

In [11]:
masks3, flows2, styles2, diams2 = model.eval(easy, channels=[0, 0],
                                         diameter=12,
                                         #cellprob_threshold=0,
                                         #flow_threshold=.4
)

viewer.add_labels(masks3, name='nuclei model')

<Labels layer 'nuclei model' at 0x2e20ee3ea40>